In [1]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import pandas as pd
import numpy as np
import blackboard
import ka
import ka_rp
import ka_br
import time
import random
import h5py

# Building a Blackboard System

We start by initializing our multi-agent system using osBrain [1].
The current implementation for verification utilizes a traditional blackboard system, where multiple agents are present, but do not act concurrently.
For the basic program, three agents will be required.

* Agent 1: Blackboard
* Agent 2: Knowledge Agent - Reactor Physics (`ka_rp`)
* Agent 3: Knowledge Agent - Blackboard Reader (`ka_br`)

The blackboard agent will retain all of the results obtained by the neutronics agent on the second abstract level of the blackboard (`level 2`).
`level 2` contains raw data in the form of a nested dictionary. 
This data consists of the design variables (height, smear, and plutonium fraction) and objectives (keff, void coefficient, Doppler coefficient, plutonium fraction).

The `ka_rp` simulates a reactor core using a surrogate model, where the design variables are used to predict a set of objective functions.
The surrogate model is generated from a set of high-fidelity MCNP solutions, using the `LinearNDinterpolator` from `scipy`, which simply performs multi-dimensional linear interpolations [2][3].
To determine the objective functions, the design variables are selected randomly between the minimum and maximum values based on previously performed resarch [4].

The `ka_br` examines `level 2` of the blackboard to determine if all of the objective variables are within some desired range.
If the `ka_br` finds a core which meets the objective functions, it will place the core name on the first level of the blackboard (`level 1`).
This will trigger the end of the simulation.

## The Problem

This simple implementation is meant to demonstrate the major aspects of a blackbaord system, and show how multiple agents are able to communicate.
To demonstrate this, we examine a simplified SFR core optimization problem.
For this problem, we seek core designs that that fulfill a set of objectives.
The design variables include the fuel height (50cm - 80cm), fuel smear (0.5 - 0.7), and the fraction of fissile material that is plutonium (0% - 100%).
The objectives for this problem are keff (1.02 - 1.07), void coefficient (-200 - -75), Doppler coefficient (-1.0 - -0.6), and the plutonium content (0% - 60%).
This example mearly seeks to find a solution within these ranges.

## Agent Initialization

We start off by initializing the blackboard, `ka_rp` and `ka_br` agents.
The two knowledge agents are initialized using the `connect_agent` function.
This connects each agent to the blackboard and connects the three different types of communication that are required to interact with the blackboard.
The blackboard is the only agent that knowledge agents (KAs) interact with.
KAs never talk with other KAs, and only pass information to each other by writing to the blackboard. 
This ia a fundamental property of the blackboard systems, even in a multi-agent environment.
Three channels of communication are used in the blackboard system: writer, trigger, and execute.

The writer channel is a request-reply communication channel, which allows the KA to continually request if the blackboard is currently being written to.
If it is, the blackboard informs the KA that another KA is writing and it must wait.
The KA will then wait for a second and request the state of the blackboard again, until it is able to write its information.
For the traditional blackboard, only one knowledge agent is active at a time, and as such it should never encounter a situation where it will be denied access to the blackboard.

The trigger channel contains two communication channels, a publish-subscribe channel and a push-pull channel.
The blackboard initiates the publish-subscribe channel, which allows the blackboard to publish a trigger event.
The trigger event is used to determine which KA will be selected for execution in the next step.
The KAs initiate the push-pull channel, which allows them respond to the trigger publication with their associated trigger value.
Trigger values will change for some KAs depending on how the problem is progressing.
For the traditional blackboard scheme, the trigger value for the `ka_rp` will always be 1, and the trigger value for the `ka_br` will be 0 if no solution is viable and 10 if a solution is viable.

The executor channel is a push-pull channel, where the blackboard will inform a KA that they have been selected for execution.
The KA who was executed performs their desired actions and will write their results to the blackboard afterwards.
We then enter a loop until the KA has finished their action for the traditional blackboard system.
Otherwise, the blackboard could continue on with the process and send another trigger event.

Along with the communication channels, we set a couple of desired variables for our KAs.
For the `ka-rp` agent no attributes are set, however for the `ka-br`, we set the ranges for our objective functions.
These ranges will determine how fast the solution will be found.

For the blackboard, the entry type for each abstract level is required to be set.
`level 1` requires that the dictionary entry have a key labed `valid`, where the value takes on a `bool` data type.
Similarly, `level 1` requires a `reactor parameters` key, where the value is itself a dictionary.
The nested dictionary for `reactor parameters` has keys for all of the design variables and objective functions, where each value is simply a `float` data type.

It is important that each KA has the correct format for adding to the blackboard.
If the keys or datatypes for each entry are not correct, the entry will not be added to the blackboard.

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=blackboard.Blackboard)
ka_rp = run_agent(name='ka_rp', base=ka_rp.KaRp_verify)
ka_br = run_agent(name='ka_br', base=ka_br.KaBr_lvl2)

bb.add_abstract_lvl(1, {'valid': bool})
bb.add_abstract_lvl(2, {'reactor parameters': {'height': float, 'smear': float, 'pu_content': float, 'keff': float, 'void_coeff': float, 'doppler_coeff': float}})

def connect_agent(agent, bb):
    agent.add_blackboard(bb)
    agent.connect_writer()
    agent.connect_trigger()
    agent.connect_executor()
    if 'rp' in agent.get_attr('name'):
         pass
    elif 'br' in agent.get_attr('name'):
        agent.set_attr(desired_results={'keff': (1.02, 1.07), 'void_coeff': (-200, -75), 'doppler_coeff': (-1.0,-0.6), 'pu_content': (0, 0.6)})

connect_agent(ka_rp, bb)
connect_agent(ka_br, bb)

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:17349 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:17349
INFO [2020-03-20 13:11:32.586098] (bb): BB connected writer to ka_rp
INFO [2020-03-20 13:11:32.589345] (ka_rp): Agent ka_rp connected writer to BB
INFO [2020-03-20 13:11:32.596397] (ka_rp): Agent ka_rp connected executor to BB
INFO [2020-03-20 13:11:32.608747] (bb): BB connected writer to ka_br
INFO [2020-03-20 13:11:32.610947] (ka_br): Agent ka_br connected writer to BB
INFO [2020-03-20 13:11:32.618609] (ka_br): Agent ka_br connected executor to BB
INFO [2020-03-20 13:11:36.256068] (bb): Determining which KA to execute
INFO [2020-03-20 13:11:36.258501] (bb): Selecting agent ka_rp (TV: 1.0) to execute (TE: 1)
INFO [2020-03-20 13:11:36.260083] (ka_rp): Core design variables determined: {'height': 51.74, 'smear': 56.09, 'pu_content': 1.0}
INFO [2020-03-20 13:11:36.260395] (bb): Writing blackboard to archive
INFO [2020-03-20 13:11:36.263797] (bb): Finished writing 

# Running the Blackboard System

The blackboadr system is controlled in the following box, where we denote a loop of the basic blackboard functions.
osBrain requires that the blackboard sytsem be run externally from the blackboard agent.
If the loop seen below is implemented into a function within the blackboard, we get errors associated with blackboard communication.
This is due to the fact that each agent is a separate process, and when the blackboard runs a method, it consumes the entire process, preventing the blackboard from processing incoming messages [1].
Currently, we avoid this problem by making an `unsafe` call to our `wait_for_ka` action.
This runs the `wait_for_ka` action in a seperate thread from the `bb`, while this may cause unusual errors it was deemed satisfactory for the verification case.
Future iterations of the blackboard system may split the blackboard into a controller and blackboard segment to separate the communication and administration portions.

Below we see a simple loop which iterates over the three major steps associated with the blackboard: trigger, controller, and execute.
As noted previously, the trigger publishes a request for trigger values, when the agents have responded the controller is initiated.
The controller examines all of the KA's trigger values and selects the knowledge agent with the highest trigger values.
Executor sends a message the the KA with the highest trigger value and tells it to execute its action.

This process will allow us to examine how the lines of communication work, and ensure that the blackboard is functioning properly.
Please note that you will see the lines of communication in the box above this.

In [3]:
for i in range(100):
    bb.publish_trigger()
    bb.controller()
    bb.send_executor()
    bb.unsafe.wait_for_ka()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))
    if bb.get_attr('abstract_lvls')['level 1'] != {}:
        break

print(bb.get_attr('abstract_lvls')['level 1'])
ns.shutdown()

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12
Finished trigger event: 13
Finished trigger event: 14
Finished trigger event: 15
Finished trigger event: 16
Finished trigger event: 17
Finished trigger event: 18
Finished trigger event: 19
Finished trigger event: 20
Finished trigger event: 21
{'core_[77.0, 51.04, 0.57]': {'valid': True}}


## Optimal Solution

Once the algorithm has been completed, we can examine the solution that was accepted.
For this we can thake the core name printed above and pull it's information from the H5 archive that the blackboard created [5].
Below, we examine all of the reactor paramters associated with the optimal core.

In [4]:
h5 = h5py.File('bb_archive.h5', 'r')
for k,v in h5['level 2']['core_[77.0, 51.04, 0.57]']['reactor parameters'].items():
    print(k,v[0])
h5.close()


doppler_coeff -0.6963270401538667
height 77.0
keff 1.0275682374687147
pu_content 0.57
smear 51.04
void_coeff -108.9262973536167


# Behind the Scenes

The previous two aspects have shown off the blackboard and its interactions with the two types of knowledge agents, but from the outside, the knowledge agents act like a black-box; we tell the agent to run and it returns some value to the blackboard.
The glean a better understanding of the blackboard system as a whole, an more in-depth discussion of the knowledge agents inner workings is required.

## Reactor Physics Knoweldge Agent

The `ka_rp` performs three steps when it is executed: determine a set of design variables, calculate the objective functions, and write the results to the blackbaord.
Determing the design variables is performed stochastically, where each variable is selected within its predefined range.
This core design is fed to the next function, which is to calculate the objective functions.
Objective functions are calculated using a surrogate model based on previously compiled data [2].
Given the core design, a set of objective functions are returned.
Both the design variables and objective functions are then written to the blackboard on `level 2`, where all raw data concerning the over-arching problem is held.

## Blackboard Reader Knowledge Agent

Th `ka_br` reads the blackboard during each trigger event to determine if any solutions is viable.
If a solution is viable, it's trigger value is set such that it will be selected over the `ka_rp`.
`ka_br` then takes the solution and places it's name on `level_1`, indicating that it has met our conditions.
Where solutions should fall within the following ranges, where units are seen in [].

* k-eigenvalue: 1.02 - 1.07
* Void Coefficient: -200 - -75 [pcm/%void]
* Doppler Coefficient: -1.0 - -0.6 [pcm/K]
* Pu Fraction: 0 - 60 [%]

## Blackboard Agent

The blackboard is a unique agent which holds the all of the information obtained by the `ka_rp` agent, and information that has been updated by `ka_br`.
This information is stored in two separate abstract levels as hinted at in the previous sections.

Abstract level 2 (`level 2`) contains data from the `ka_rp` in the form of a dictionary, whose keys are the core name (in the form of `Core_[design_variables])`).
The each core contains a dictionary for the reactor parmaters (`reactor paremters`), which is in turn a dictionary for all of the objective and design variables.

Abstract level 1 (`level 1`) stores core designs that meet the requirements layed out in the previous section.
Currently, when a this `level 1` has an entry, the solution is terminated, however, one can imagine a case where we want to find multiple solutions which meet our objectives.

## Concluding Remarks

The blackboard system from this example is able to find a solution which falls within our range relatively quickly (typically within 20 iterations).
If we were to narrow our margins for a desirable solution, it is imaginable that this time would drastically increase since we are currently applying a brute-force stocastic algorithm to find optimal solutions.

# References

[1] osBrain v0.6.5, (2019), GitHub repository, https://github.com/opensistemas-hub/osbrain.

[2] C.J. Werner,  et al.,  “MCNP6.2 Release Notes”,  Los Alamos National Laboratory,  reportLA-UR-18-20808 (2018).

[3] E. Jones, E. Oliphant, P. Peterson, et al, “SciPy: Open Source Scientific Tools for Python”,http://www.scipy.org/ (2001).

[4] R. Stewart and T.S. Palmer, "Utilizing a Reduced-Order Model and Physical Programming for Preliminary Reactor Design Optimization," PHYSOR-2020, Cambridge, UK, 2020.

[5] The HDF Group. "Hierarchical Data Format, version 5,"  http://www.hdfgroup.org/HDF5/, (1997).
